# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [31]:
# Importing libraries
import numpy as np  # numerical computing library
import pandas as pd # panel data library
import requests     # http requests library
import xlsxwriter   # Excel documents writing library
import math         # mathematical functions library

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv") # Obs: there are 5 stocks that have dual shares, that's why there are 505 stocks
stocks

stocks = pd.read_csv("Meat_stocks.csv")

stocks

,Ticker
0,BRFS
1,HRL
2,IBA
3,TSN


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN # Sometimes you need to ´restart´ the Jupyter Kernel to run this line

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [6]:
symbol = "AAPL" # This is the symbol for Apple stocks
symbol = "BRFS"
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}' # this is the url for the sandbox api
# The next line could take a few seconds
data = requests.get(api_url).json() # .json() make a type python dictionary object
print(data)
print(type(data))

{'symbol': 'BRFS', 'companyName': 'BRF S.A. - ADR', 'primaryExchange': 'C OI,SW EGOYNCEA X CKHR.KETNN', 'calculationPrice': 'close', 'open': None, 'openTime': None, 'openSource': 'ioflifca', 'close': None, 'closeTime': None, 'closeSource': 'lafcfoii', 'high': None, 'highTime': 1629724648232, 'highSource': 'naie d y1etcel5rmdi uep', 'low': None, 'lowTime': 1613489242526, 'lowSource': 'i uyme5rn1pedit  caelde', 'latestPrice': 4.3, 'latestSource': 'Close', 'latestTime': 'December 31, 2020', 'latestUpdate': 1650973935414, 'latestVolume': None, 'iexRealtimePrice': 4.257, 'iexRealtimeSize': 103, 'iexLastUpdated': 1660214921519, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 4.32, 'previousVolume': 1633931, 'change': -0.07, 'changePercent': -0.01673, 'volume': None, 'iexMarketPercent': 0.02273742133284901, 'iexVo

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [7]:
price = data['latestPrice']
market_cap = data['marketCap']
print(price)

4.3


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [8]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [9]:
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ], index = my_columns
        ), ignore_index = True
)

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,BRFS,4.3,3438413389,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [10]:
final_dataframe = pd.DataFrame(columns=my_columns)
i = 0
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}' # this is the url for the sandbox api
    data = requests.get(api_url).json() # .json() make a type python dictionary object
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ], index = my_columns
        ), ignore_index = True
    )
    print(i)
    i=i+1

0
1
2
3


In [11]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,BRFS,4.20,3560448773,N/A
1,HRL,48.48,25680556565,N/A
2,IBA,45.40,2292723991,N/A
3,TSN,65.28,24511195934,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [12]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [25]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings =[]
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])
    

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
          final_dataframe = final_dataframe.append(
              pd.Series(
              [
                  symbol,
                  data[symbol]['quote']['latestPrice'],
                  data[symbol]['quote']['marketCap'],
                  'N/A'
              ], index = my_columns
              ), ignore_index = True
          )

final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,BRFS,4.40,3542656589,N/A
1,HRL,47.52,25893430612,N/A
2,IBA,46.49,2290032894,N/A
3,TSN,66.91,24309454675,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [28]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print('Please enter an integer')

Enter the value of your portfolio:10000
10000.0


In [29]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,BRFS,4.40,3542656589,568
1,HRL,47.52,25893430612,52
2,IBA,46.49,2290032894,53
3,TSN,66.91,24309454675,N/A


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [30]:
writer = pd.ExcelWriter('Recomended_trades_Meat_Stocks_of_NYSE.xlsx',writer='xlsxwriter')
final_dataframe.to_excel(writer, 'Recomended Trades', index=False)

TypeError: __init__() got an unexpected keyword argument 'writer'

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: